In [ ]:
import pandas as pd

data = pd.read_csv("/content/target_product_search.csv")
data.head()

,Unnamed: 0,store_id,product_id,title,brand,price,availability,rating,reviews,url,category,keyword,load_date,load_timestamp,record_date
0,0,2822,93351038,12&#34; Lit Monster Boo Blow Mold Halloween De...,NaN,$10.00,NaN,3.97,90.0,https://www.target.com/p/12-34-lit-monster-boo...,NaN,halloween_decor,2025-10-15,2025-10-15T15:38:22.647506,2025-10-15T15:38:24.094773
1,1,2822,81063375,11&#34; French Bulldog Skeleton Halloween Deco...,NaN,$18.00,NaN,4.77,331.0,https://www.target.com/p/french-bulldog-skelet...,NaN,halloween_decor,2025-10-15,2025-10-15T15:38:22.647506,2025-10-15T15:38:24.094773
2,2,2822,89449319,Seasonal Visions Animated Spell-Speaking Witch...,NaN,$355.73,NaN,0.00,0.0,https://www.target.com/p/seasonal-visions-anim...,NaN,halloween_decor,2025-10-15,2025-10-15T15:38:22.647506,2025-10-15T15:38:24.094773
3,3,2822,89600213,11&#39; Stretchy Spiderweb Halloween Decorativ...,NaN,$5.00,NaN,NaN,NaN,https://www.target.com/p/stretchy-spiderweb-gl...,NaN,halloween_decor,2025-10-15,2025-10-15T15:38:22.647506,2025-10-15T15:38:24.094773
4,4,2822,81063458,20&#34; Plush Spider Halloween Decorative Prop...,NaN,$6.00,NaN,NaN,NaN,https://www.target.com/p/20-34-plush-spider-bl...,NaN,halloween_decor,2025-10-15,2025-10-15T15:38:22.647506,2025-10-15T15:38:24.094773


In [ ]:
data.keyword.unique()

array(['christmas_decor'], dtype=object)

In [ ]:
data = data[data["keyword"]=='christmas_decor']

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 600 entries, 200 to 1849
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      600 non-null    int64  
 1   store_id        600 non-null    int64  
 2   product_id      600 non-null    int64  
 3   title           600 non-null    object 
 4   brand           0 non-null      float64
 5   price           600 non-null    object 
 6   availability    0 non-null      float64
 7   rating          482 non-null    float64
 8   reviews         482 non-null    float64
 9   url             600 non-null    object 
 10  category        0 non-null      float64
 11  keyword         600 non-null    object 
 12  load_date       600 non-null    object 
 13  load_timestamp  600 non-null    object 
 14  record_date     600 non-null    object 
dtypes: float64(5), int64(3), object(7)
memory usage: 75.0+ KB


In [ ]:
data.to_csv("chirstmas_decor_data.csv")

In [ ]:
!pip install sdv==1.28.0

In [ ]:
"""
train_sdv_model.py
Trains a GaussianCopulaSynthesizer on Target-style product search data
and saves the model + metadata for use in Cloud Functions.
"""

import pandas as pd
import numpy as np
import random
from sdv.single_table import GaussianCopulaSynthesizer
from sdv.metadata import SingleTableMetadata

# -------------------------------------------------------
# 1️⃣ Create base training dataset (Target-style structure)
# -------------------------------------------------------
print("🔹 Creating base product dataset...")

base_df = data

print(base_df.head())

# -------------------------------------------------------
# 2️⃣ Generate metadata and fit the model
# -------------------------------------------------------
print("🔹 Generating metadata & training SDV model...")


metadata = SingleTableMetadata()
metadata.detect_from_dataframe(base_df)
synth = GaussianCopulaSynthesizer(metadata)
synth.fit(base_df)

# -------------------------------------------------------
# 3️⃣ Save model + metadata for deployment
# -------------------------------------------------------
metadata.save_to_json("target_product_metadata.json")
synth.save("target_product_synth.pkl")

print("✅ Training complete.")
print("📂 Saved files:")
print("  - target_product_metadata.json")
print("  - target_product_synth.pkl")

# -------------------------------------------------------
# 4️⃣ Test sample generation
# -------------------------------------------------------
sample = synth.sample(5)
print("\n🔸 Example Synthetic Samples:")
print(sample.head())


🔹 Creating base product dataset...
     Unnamed: 0  store_id  product_id  \
200         200      2822    94339971   
201         201      2822  1006589524   
202         202      2822    90516952   
203         203      2822    94339949   
204         204      2822    94339968   

                                                 title  brand   price  \
200  Christmas Featherly Friends &#34;Kelk﻿ka&#34; ...    NaN   $5.00   
201  6 Pieces Christmas Decorations with LED Light,...    NaN  $99.99   
202  3pc Lit Ceramic Buildings Decorative Christmas...    NaN  $15.00   
203  Christmas Featherly Friends &#34;Dash&#34; Dec...    NaN   $5.00   
204  4.25&#34; 3pk Christmas Featherly Friends Deco...    NaN  $15.00   

     availability  rating  reviews  \
200           NaN    5.00     10.0   
201           NaN    0.00      0.0   
202           NaN    3.69    116.0   
203           NaN     NaN      NaN   
204           NaN    2.00     13.0   

                                                  

/usr/local/lib/python3.12/dist-packages/sdv/single_table/base.py:168: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
/usr/local/lib/python3.12/dist-packages/sdv/single_table/base.py:134: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


✅ Training complete.
📂 Saved files:
  - target_product_metadata.json
  - target_product_synth.pkl

🔸 Example Synthetic Samples:
   Unnamed: 0  store_id  product_id  \
0     2796832      3223    94345205   
1     8452924      3304    94339968   
2     3865118      1229    94340017   
3     7152446      3363    94339949   
4     4264306      1898  1006077663   

                                               title  brand   price  \
0  Christmas Ceramic Standing Pink Deer Decor - W...    NaN   $6.00   
1  10&#34; Christmas Fabric Tree Decor Green - Wo...    NaN  $15.00   
2  8pk Christmas Poly Figural Decor Set Birds/Bun...    NaN  $15.00   
3  4.25&#34; 3pk Christmas Featherly Friends Deco...    NaN  $15.00   
4  20&#34; Christmas Single Bow Decor Red - Wonde...    NaN  $83.99   

   availability  rating  reviews  \
0           NaN     NaN     13.0   
1           NaN    1.98      NaN   
2           NaN     NaN      4.0   
3           NaN    4.32     47.0   
4           NaN    4.64     53

In [ ]:
display(data.head())
display(data.info())

,Unnamed: 0,store_id,product_id,title,brand,price,availability,rating,reviews,url,category,keyword,load_date,load_timestamp,record_date
200,200,2822,94339971,Christmas Featherly Friends &#34;Kelk﻿ka&#34; ...,NaN,$5.00,NaN,5.00,10.0,https://www.target.com/p/9ct-80mm-christmas-fl...,NaN,christmas_decor,2025-10-18,2025-10-18T02:57:55.120610,2025-10-18T02:59:47.419023
201,201,2822,1006589524,"6 Pieces Christmas Decorations with LED Light,...",NaN,$99.99,NaN,0.00,0.0,https://www.target.com/p/6-pieces-christmas-de...,NaN,christmas_decor,2025-10-18,2025-10-18T02:57:55.120610,2025-10-18T02:59:47.419023
202,202,2822,90516952,3pc Lit Ceramic Buildings Decorative Christmas...,NaN,$15.00,NaN,3.69,116.0,https://www.target.com/p/3pc-lit-ceramic-build...,NaN,christmas_decor,2025-10-18,2025-10-18T02:57:55.120610,2025-10-18T02:59:47.419023
203,203,2822,94339949,Christmas Featherly Friends &#34;Dash&#34; Dec...,NaN,$5.00,NaN,NaN,NaN,https://www.target.com/p/christmas-fabric-bird...,NaN,christmas_decor,2025-10-18,2025-10-18T02:57:55.120610,2025-10-18T02:59:47.419023
204,204,2822,94339968,4.25&#34; 3pk Christmas Featherly Friends Deco...,NaN,$15.00,NaN,2.00,13.0,https://www.target.com/p/3pk-christmas-fabric-...,NaN,christmas_decor,2025-10-18,2025-10-18T02:57:55.120610,2025-10-18T02:59:47.419023


<class 'pandas.core.frame.DataFrame'>
Index: 600 entries, 200 to 1849
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      600 non-null    int64  
 1   store_id        600 non-null    int64  
 2   product_id      600 non-null    int64  
 3   title           600 non-null    object 
 4   brand           0 non-null      float64
 5   price           600 non-null    object 
 6   availability    0 non-null      float64
 7   rating          482 non-null    float64
 8   reviews         482 non-null    float64
 9   url             600 non-null    object 
 10  category        0 non-null      float64
 11  keyword         600 non-null    object 
 12  load_date       600 non-null    object 
 13  load_timestamp  600 non-null    object 
 14  record_date     600 non-null    object 
dtypes: float64(5), int64(3), object(7)
memory usage: 75.0+ KB


None